In [ ]:
# HIDDEN
# For Tables reference see http://data8.org/datascience/tables.html
# This useful nonsense should just go at the top of your notebook.
from datascience import *
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')
plots.rc('lines', linewidth=2, color='r')
from ipywidgets import interact
# datascience version number of last run of this notebook
version.__version__

In [ ]:
# The following is a bunch of code to make data presentation simple.  Skip it for reading.
import sys
sys.path.append("..")
from timetable import TimeTable

import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

import power
import datetime
from os import path

In [ ]:
def addsite(site, sites, models, model_perr):
    try :
        v = ucb.view(site, 'Building_Power_Demand_Sensor')
        raw_ts, md = v.getday(day)
        ts = raw_ts.snap(np.arange(0, 24, 0.25), 'kW')
    except :
        print(site, "failed to fetch")
        return
    p_5 = power.low(ts['kW'])
    p_ave = np.mean(ts['kW'])
    p_95 = power.high(ts['kW'])
    try :
        params, perr = power.loadfit(ts, p_5, p_95)
        ts['model'] = ts.apply(power.loadmodel(*params), 'hour')
        sites[site] = ts
        summary = [site, p_5, p_ave, p_95] + list(params)
        print(summary)
        models.append(summary)
        model_perr.append([site] + list(perr))
    except :
        print(site, "Fetched but model failed")

In [ ]:
day = "2019-04-09"

In [ ]:
ucb = power.CampusPower('https://campus-export.xbos.io')

In [ ]:
sites = {}
models = Table(['site', 'p5', 'pave', 'p95', 't1', 't2', 't3', 't4', 'p_lo', 'p_hi'])
model_perr = Table(['site', 't1', 't2', 't3', 't4', 'p_lo', 'p_hi'])

In [ ]:
for site in ucb.sites() :
    if not site in models['site'] :
        addsite(site, sites, models, model_perr)

In [ ]:
models

In [ ]:
models.num_rows

In [ ]:
models['hi/lo'] = models['p_hi']/models['p_lo']

In [ ]:
models.sort('pave', descending=True).show()

In [ ]:
models.select(['t1', 't2', 't3', 't4']).hist(bins=48)

In [ ]:
models.select('hi/lo').hist(bins=20)

In [ ]:
for site in models.sort('pave', descending=True)['site']:
    sites[site].plot('hour')
    plots.title(site)